In [1]:
from sklearn.datasets import fetch_openml
from spn.algorithms.Inference import log_likelihood
from spn.algorithms.LearningWrappers import learn_parametric
from spn.structure.Base import Context
from spn.structure.leaves.parametric.Parametric import Categorical, Gaussian, Bernoulli
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

bunch = fetch_openml(data_id = 183)
print ("%s rows" % (len(bunch.frame)))
print (bunch.frame.shape)
bunch.frame.head()

4177 rows
(4177, 9)


,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Class_number_of_rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [2]:
frame = bunch.frame
frame.Class_number_of_rings = frame.Class_number_of_rings.astype(int)
values, text = pd.factorize(frame.Sex, sort=True)
frame.Sex = values

names = frame.columns.tolist()
categorical = [False]*len(names)
categorical[0] = True
ptypes = [Categorical if categorical[i] else Gaussian for i, name in enumerate(names)]

train, test = train_test_split(frame.to_numpy(), random_state = 0)
print ("%d in train, %d in test" % (len(train), len(test)))

3132 in train, 1045 in test


In [3]:
net = learn_parametric(train, 
                       ds_context = Context(parametric_types=ptypes).add_domains(train), 
                       rows = "gmm", ohe=True,
                       min_instances_slice = len(train) / 100)

/home/saurabh/miniconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
/home/saurabh/miniconda3/lib/python3.7/site-packages/sklearn/cross_decomposition/_pls.py:98: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
/home/saurabh/miniconda3/lib/python3.7/site-packages/sklearn/cross_decomposition/_pls.py:98: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)


In [5]:
import csi2
from spn.structure.Base import get_nodes_by_type, Product



def format_condition(condition):
    try:
        a, sign, b = condition.split(" ")
    except ValueError as e:
        raise ValueError("Some nodes don't have conditions. reduce min_impurity_decrease")
    if sign in ('==', '!='):
        assert a == 'Sex'
        v = text[int(float(b))]
        return ("%s %s %s" % (a, sign, v))
    else:
        b = float(b)
        return ("%s %s %.2f" % (a, sign, b))

print ('#product nodes = ', len(get_nodes_by_type(net, (Product))))

csi2.annotate_spn(net, names, categorical, 
             min_impurity_decrease = 0.05, max_depth = 2)
rules = csi2.context_specific_independences(net, instance_threshold = 0)


rules = csi2.context_specific_independences(net, instance_threshold = 0)
csis = []
ac = []
cc = []
for i, rule in enumerate(rules):
    antecedent, consequent, *scores = rule
    A = csi2.format_antecedent(antecedent, format_condition)
    C = csi2.format_consequent(consequent)
    a_count = csi2.antecedent_count(A)
    c_count = csi2.consequent_count(C)
    ac.append(a_count)
    cc.append(c_count)
    csis.append ("{%s} => {%s} | %.2f, %.2f, %d | %d %d" % (A, C, *scores, a_count, c_count))
print ("%d, %.2f, %.2f" % (len(csis), np.mean(ac), np.mean(cc)))

rules = csi2.context_specific_independences(net, instance_threshold = len(train) / 20,
                                           precision_threshold = 0.7, recall_threshold = 0.7)
csis = []
ac = []
cc = []
for i, rule in enumerate(rules):
    antecedent, consequent, *scores = rule
    A = csi2.format_antecedent(antecedent, format_condition)
    C = csi2.format_consequent(consequent)
    a_count = csi2.antecedent_count(A)
    c_count = csi2.consequent_count(C)
    ac.append(a_count)
    cc.append(c_count)
    csis.append ("{%s} => {%s} | %.2f, %.2f, %d | %d %d" % (A, C, *scores, a_count, c_count))
print ("%d, %.2f, %.2f" % (len(csis), np.mean(ac), np.mean(cc)))

#product nodes =  194
194, 11.31, 7.00
4, 4.25, 2.00


In [6]:
csis

['{[Whole_weight <= 0.71] & [Whole_weight > 0.38]} => {(Sex), (Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Class_number_of_rings)} | 0.83, 0.87, 724 | 2 2',
 '{[Whole_weight <= 0.71] & [Whole_weight <= 0.38] & [Sex == I]} => {(Sex), (Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Class_number_of_rings)} | 0.83, 0.89, 416 | 3 2',
 '{[Whole_weight > 0.71] & [Class_number_of_rings <= 12.50] & [((Height <= 0.16) & (Sex == M)) | (Height > 0.16)]} => {(Sex), (Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Class_number_of_rings)} | 0.81, 0.83, 846 | 5 2',
 '{[Whole_weight <= 0.71] & [Whole_weight > 0.38] & [(Height <= 0.18) & (Height > 0.10)] & [(Class_number_of_rings <= 9.50) & (Whole_weight <= 0.70)] & [Shell_weight > 0.13]} => {(Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight), (Class_number_of_rings)} | 0.83, 0.84, 340 | 7 2']

In [5]:
from spn.algorithms.Inference import log_likelihood
import numpy as np

print ("test log-likelihood = %.2f" % np.mean(log_likelihood(net, test)))

test log-likelihood = 18.99
